In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import sagemaker

In [2]:
session = boto3.Session(profile_name='default',region_name='us-east-2')

In [3]:
sagemaker_session = sagemaker.Session(boto_session=session)

In [4]:
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

In [5]:
bucket

'sagemaker-us-east-2-113516067754'

In [6]:
data_dir='../aws_test'

In [7]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix='test_data')

In [2]:
from sklearn.datasets import load_iris

In [3]:
iris=load_iris()

In [4]:
X=iris['data']
y=iris['target']

In [5]:
X.shape

(150, 4)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
lgr=LogisticRegression()

In [8]:
lgr.fit(X,y)

LogisticRegression()

In [9]:
accuracy_score(lgr.predict(X),y)

0.9733333333333334

In [21]:
import pickle

In [22]:
pickle.dump(lgr,open('model_lgr.pkl','wb'))

In [48]:
input_data = sagemaker_session.upload_data(path='./model.tar.gz', bucket=bucket, key_prefix='model_lgr')

In [8]:
input_data='s3://sagemaker-us-east-2-113516067754/model_lgr/model.tar.gz'

# Deployment

In [6]:
from sagemaker.sklearn.model import SKLearnModel

In [21]:
import os, json

In [31]:
sklearn.__version__

'0.23.2'

In [7]:
role

'arn:aws:iam::113516067754:role/service-role/AmazonSageMaker-ExecutionRole-20200513T111076'

In [50]:
sklearn_model=SKLearnModel(model_data=input_data,
                             role=role,
                             entry_point="lgr_aws.py",
                             framework_version="0.23-1",
                            source_dir="aws_test",
                            dependencies=['./aws_test/model']
                            )

In [51]:
predictor=sklearn_model.deploy(initial_instance_count=1,instance_type='ml.t2.medium')

-----------------!

In [19]:
from sagemaker.predictor import json_serializer, json_deserializer

In [20]:
predictor.serializer=json_serializer
predictor.deserializer=json_deserializer

In [24]:
predictor.content_type='application/json'
predictor.accept='application/json'

In [29]:
data={'X':X[0:1].tolist()}

In [30]:
json.dumps(data)

'{"X": [[5.1, 3.5, 1.4, 0.2]]}'

In [25]:
pred=predictor.predict(data)

In [52]:
predictor.delete_endpoint()

In [54]:
load_iris.__module__

'sklearn.datasets._base'

# Local Mode

In [12]:
from sagemaker.local import LocalSession

In [10]:
from sagemaker.sklearn.model import SKLearnModel

In [13]:
sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

In [14]:
role = 'arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001'

In [24]:
sklearn_model=SKLearnModel(model_data='file://./model.tar.gz',
                             role=role,
                             entry_point="lgr_aws.py",
                             framework_version="0.23-1",
                            source_dir="aws_deploy",
                            dependencies=['./aws_deploy/model']
                            )

In [26]:
sklearn_model.deploy(initial_instance_count=1,instance_type='local')

Using the short-lived AWS credentials found in session. They might expire while running.
Attaching to tmps3rq3cpu_algo-1-hhv9q_1
algo-1-hhv9q_1  | 2021-01-11 20:46:20,830 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-hhv9q_1  | 2021-01-11 20:46:20,837 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-hhv9q_1  | 2021-01-11 20:46:20,839 INFO - sagemaker-containers - nginx config: 
algo-1-hhv9q_1  | worker_processes auto;
algo-1-hhv9q_1  | daemon off;
algo-1-hhv9q_1  | pid /tmp/nginx.pid;
algo-1-hhv9q_1  | error_log  /dev/stderr;
algo-1-hhv9q_1  | 
algo-1-hhv9q_1  | worker_rlimit_nofile 4096;
algo-1-hhv9q_1  | 
algo-1-hhv9q_1  | events {
algo-1-hhv9q_1  |   worker_connections 2048;
algo-1-hhv9q_1  | }
algo-1-hhv9q_1  | 
algo-1-hhv9q_1  | http {
algo-1-hhv9q_1  |   include /etc/nginx/mime.types;
algo-1-hhv9q_1  |   default_type application/octet-stream;
algo-1-hhv9q_1  |   access_log /dev/stdout combined;
algo-1-hhv9q_